In [1]:
import requests
import datetime
import time
import json

from IPython.display import display, Markdown, JSON

In [3]:
with open('./API_TOKEN.txt', 'r') as tokenfile:
    TOKEN = tokenfile.read()

request_headers = {
    'Authorization': 'token '+TOKEN,
    'Accept': 'application/vnd.github.mercy-preview+json'
}

def get_repos():
    # Fetch data from GitHub API via HTTPS request
    response = []
    for p in range(1, 6):
        query = 'https://api.github.com/users/generic-github-user/repos?page='+str(p)
        data = requests.get(query, headers=request_headers).json()
    #     print(data)
        response.extend(data)
    return response

response = get_repos()
print(len(response))
JSON(response[:5])

125


<IPython.core.display.JSON object>

In [4]:
repo_trees = {}

In [8]:
def find_file(repo, filename, cache=True):
    title = repo['name']
    if cache and title in repo_trees and 'tree' in repo_trees[title]:
        tree = repo_trees[title]
    else:
#         Request the file tree of the repository's default branch (as of the latest commit)
        branch = repo['default_branch']
        request_path = '/'.join([repo['url']] + ['git', 'trees', f'{branch}?recursive=1'])
        print(f'Requesting data about repository {title} from {request_path}...')
        tree = requests.get(request_path, headers={'Authorization': 'token '+TOKEN}).json()
        repo_trees[title] = tree
#         Rate limit requests to avoid exceeding quota
        time.sleep(0.1)
    
#     Check if tree exists and a README file exists
    if 'tree' in tree and any(filename in f['path'] for f in tree['tree']):
        return '✅'
    else:
        return ''

def truncate(x):
    x = [x_ for x_ in x]
    num = len(x)
    for i in range(len(x)):
        if sum(map(len, x)) < 30:
            break
        else:
            x.pop()
    return x, (num-len(x))

# Convert NoneType and None/null string values to empty strings
def plain(x):
    return x if (x and x not in ['None', 'null']) else ''

# Convert a list of topic strings to a formatted list
def format_topics(r):
    topics, n = truncate(r['topics'])
    return ' '.join(f'`{t}`' for t in topics) + (f'*({n} more)*' if n else '')

# Shorten the repository description if it exceeds a set length
def format_description(r):
    desc = plain(r['description'])
    if len(desc) > 50:
        desc = desc[:50] + ('.'*3)
    return desc
    
columns = [
    ('Title', 'name', ''),
    ('Description', format_description),
    ('Topics', lambda r: format_topics(r)),
#     ('Issues', 'open_issues_count', 'issues'),
    ('Issues', lambda r: f'**[{r["open_issues_count"]}]({r["html_url"]+"/issues"})** [+]({r["html_url"]+"/issues/new"})'),
    ('README', lambda r: find_file(r, 'README.md')),
    ('License', lambda r: find_file(r, 'LICENSE')),
    ('Created', lambda r: datetime.datetime.strptime(r['created_at'], '%Y-%m-%dT%H:%M:%SZ').strftime('%b %Y')),
    ('Size', 'size', None, ' KB')
]

def format_info(x, y, z=None, w=None, r=None):
    if callable(y) and r:
        result = y(r)
    else:
        result = plain(str(r[y]).strip())
    
    if z is not None:
        url = r['html_url']
        link = url+'/'+z
        result = f'[{result}]({link})'
    if w:
        result += w
        
    return result

cache_path = './github-cache.json'
try:
    with open(cache_path, 'r') as cachefile:
        repo_trees = json.loads(cachefile.read())
except:
    print('No cache found')

In [9]:
sum(map(bool, ['test']))

1

In [10]:
def count_prop(c):
#     return sum([(p in repo and repo[p]) for repo in response])
    return sum(map(bool, [format_info(*c, r=r) for r in response if not r['fork']]))

def generate_table():
    """
    Produce a table summarizing selected attributes of each retrieved repository
    """
    
    response = get_repos()
    divider = ' | '.join(['---']*len(columns))
    dash = '-'*3
#     Build the header from the column titles and alignments
    header = ' | '.join([c[0] for c in columns]) + '\n' + ' | '.join(f':{dash}:' if c[0] in ['README', 'Issues', 'Created'] else dash for c in columns)
    content = header

    # response.sort(reverse=True, key=lambda r: r['open_issues_count'])
#     Convert the timestamp to a human-readable string
    for repo in response:
        repo['milliseconds'] = datetime.datetime.strptime(repo['created_at'], '%Y-%m-%dT%H:%M:%SZ').timestamp()
#     Sort repos
    response.sort(reverse=True, key=lambda r: r['milliseconds'])

#     Columns to ignore when calculating totals/counting attributes
    exclude_cols = ['Title', 'Issues', 'Created', 'Size']
    content += '\n' + ' | '.join(
        map(str, [str(round(count_prop(col) / len(response) * 100))+'%' if col[0] not in exclude_cols else ' ' for col in columns])
    )

#     Loop through repositories and generate rows
    for repo in response[:]:
#         Exclude forked repos
        if not repo['fork']:
            content += '\n' + ' | '.join([format_info(*col, r=repo) for col in columns])
    
#     Write output to file
    with open('./output.md', 'w', encoding='UTF-8') as outputfile:
        outputfile.write(content)
#     Cache data requested from GitHub API
    with open(cache_path, 'w') as newcache:
        json.dump(repo_trees, newcache)
        
    return content
    
content = generate_table()
Markdown(content)

Title | Description | Topics | Issues | README | License | Created | Size
--- | --- | --- | :---: | :---: | --- | :---: | ---
  | 65% | 40% |   | 81% | 26% |   |  
[project-summary](https://github.com/generic-github-user/project-summary/) | Generate a formatted list of your repositories wit... | `python` `markdown` `github-api` `tools`*(1 more)* | **[5](https://github.com/generic-github-user/project-summary/issues)** [+](https://github.com/generic-github-user/project-summary/issues/new) |  |  | Jul 2021 | 30 KB
[card-game-analysis](https://github.com/generic-github-user/card-game-analysis/) | Monte Carlo simulations of poker and other card ga... | `statistics` `analysis` `simulation`*(3 more)* | **[0](https://github.com/generic-github-user/card-game-analysis/issues)** [+](https://github.com/generic-github-user/card-game-analysis/issues/new) |  |  | Jul 2021 | 0 KB
[programming-puzzles](https://github.com/generic-github-user/programming-puzzles/) | Some creative programming challenges in the style ... | `python` `mathematics` `reasoning`*(1 more)* | **[0](https://github.com/generic-github-user/programming-puzzles/issues)** [+](https://github.com/generic-github-user/programming-puzzles/issues/new) |  |  | Jun 2021 | 32 KB
[python-snippets](https://github.com/generic-github-user/python-snippets/) | Useful bits of code | `python` `snippets` `tools` `utilities` | **[2](https://github.com/generic-github-user/python-snippets/issues)** [+](https://github.com/generic-github-user/python-snippets/issues/new) | ✅ |  | Jun 2021 | 6 KB
[self-avoiding-walks](https://github.com/generic-github-user/self-avoiding-walks/) | Simulating self-avoiding random walks on lattices ... | `simulation` `mathematics`*(4 more)* | **[0](https://github.com/generic-github-user/self-avoiding-walks/issues)** [+](https://github.com/generic-github-user/self-avoiding-walks/issues/new) |  |  | Jun 2021 | 65 KB
[coincidence](https://github.com/generic-github-user/coincidence/) | Scripts and functions for discovering mathematical... | `mathematics` `combinatorics`*(2 more)* | **[2](https://github.com/generic-github-user/coincidence/issues)** [+](https://github.com/generic-github-user/coincidence/issues/new) |  | ✅ | Jun 2021 | 3 KB
[python-experiments](https://github.com/generic-github-user/python-experiments/) | Exploring some experimental tweaks to & novel ways... | `python` `experimental`*(3 more)* | **[0](https://github.com/generic-github-user/python-experiments/issues)** [+](https://github.com/generic-github-user/python-experiments/issues/new) | ✅ | ✅ | Jun 2021 | 13 KB
[lightning-simulation](https://github.com/generic-github-user/lightning-simulation/) | Simple stochastic lightning simulation and renderi... | `lightning` `graphics`*(2 more)* | **[2](https://github.com/generic-github-user/lightning-simulation/issues)** [+](https://github.com/generic-github-user/lightning-simulation/issues/new) | ✅ | ✅ | Jun 2021 | 584 KB
[metamorphic](https://github.com/generic-github-user/metamorphic/) | Experiments with self-obfuscating code | `obfuscation` `reflection`*(4 more)* | **[11](https://github.com/generic-github-user/metamorphic/issues)** [+](https://github.com/generic-github-user/metamorphic/issues/new) |  | ✅ | Jun 2021 | 43 KB
[byzantine](https://github.com/generic-github-user/byzantine/) | An extremely open open-world sandbox exploration/c... | `game` `space-game` `sandbox-game`*(1 more)* | **[2](https://github.com/generic-github-user/byzantine/issues)** [+](https://github.com/generic-github-user/byzantine/issues/new) |  |  | Jun 2021 | 2 KB
[evo-art](https://github.com/generic-github-user/evo-art/) |  |  | **[0](https://github.com/generic-github-user/evo-art/issues)** [+](https://github.com/generic-github-user/evo-art/issues/new) |  |  | Jun 2021 | 5 KB
[text-diagrams](https://github.com/generic-github-user/text-diagrams/) | A tool for creating ASCII and/or Unicode drawings,... |  | **[19](https://github.com/generic-github-user/text-diagrams/issues)** [+](https://github.com/generic-github-user/text-diagrams/issues/new) | ✅ | ✅ | Jun 2021 | 243 KB
[particles](https://github.com/generic-github-user/particles/) | Simulating & rendering particle systems with Pytho... |  | **[0](https://github.com/generic-github-user/particles/issues)** [+](https://github.com/generic-github-user/particles/issues/new) | ✅ |  | Jun 2021 | 4 KB
[grid-based-simulation](https://github.com/generic-github-user/grid-based-simulation/) |  |  | **[2](https://github.com/generic-github-user/grid-based-simulation/issues)** [+](https://github.com/generic-github-user/grid-based-simulation/issues/new) |  |  | Jun 2021 | 6 KB
[generic-github-user](https://github.com/generic-github-user/generic-github-user/) |  |  | **[0](https://github.com/generic-github-user/generic-github-user/issues)** [+](https://github.com/generic-github-user/generic-github-user/issues/new) | ✅ |  | Jun 2021 | 8 KB
[turquoise](https://github.com/generic-github-user/turquoise/) | My programming language | `programming-language`*(3 more)* | **[52](https://github.com/generic-github-user/turquoise/issues)** [+](https://github.com/generic-github-user/turquoise/issues/new) | ✅ |  | Jun 2021 | 184 KB
[locus](https://github.com/generic-github-user/locus/) | A visuospatial esoteric programming language based... | `python` `programming-language`*(1 more)* | **[9](https://github.com/generic-github-user/locus/issues)** [+](https://github.com/generic-github-user/locus/issues/new) |  |  | May 2021 | 14 KB
[fractals](https://github.com/generic-github-user/fractals/) |  |  | **[20](https://github.com/generic-github-user/fractals/issues)** [+](https://github.com/generic-github-user/fractals/issues/new) | ✅ |  | May 2021 | 3942 KB
[cellular-automata-experiments](https://github.com/generic-github-user/cellular-automata-experiments/) |  |  | **[14](https://github.com/generic-github-user/cellular-automata-experiments/issues)** [+](https://github.com/generic-github-user/cellular-automata-experiments/issues/new) | ✅ |  | Apr 2021 | 89 KB
[diagrammer](https://github.com/generic-github-user/diagrammer/) |  |  | **[1](https://github.com/generic-github-user/diagrammer/issues)** [+](https://github.com/generic-github-user/diagrammer/issues/new) | ✅ |  | Apr 2021 | 21 KB
[roulette-curves](https://github.com/generic-github-user/roulette-curves/) |  |  | **[0](https://github.com/generic-github-user/roulette-curves/issues)** [+](https://github.com/generic-github-user/roulette-curves/issues/new) |  |  | Apr 2021 | 8 KB
[primes](https://github.com/generic-github-user/primes/) |  |  | **[0](https://github.com/generic-github-user/primes/issues)** [+](https://github.com/generic-github-user/primes/issues/new) |  |  | Apr 2021 | 0 KB
[crossword-generator](https://github.com/generic-github-user/crossword-generator/) |  |  | **[0](https://github.com/generic-github-user/crossword-generator/issues)** [+](https://github.com/generic-github-user/crossword-generator/issues/new) |  |  | Apr 2021 | 0 KB
[decision-tree-experiments](https://github.com/generic-github-user/decision-tree-experiments/) |  |  | **[17](https://github.com/generic-github-user/decision-tree-experiments/issues)** [+](https://github.com/generic-github-user/decision-tree-experiments/issues/new) | ✅ |  | Apr 2021 | 41 KB
[music-generation-experiments](https://github.com/generic-github-user/music-generation-experiments/) | Playing around with MIDI & music theory |  | **[4](https://github.com/generic-github-user/music-generation-experiments/issues)** [+](https://github.com/generic-github-user/music-generation-experiments/issues/new) | ✅ |  | Apr 2021 | 107 KB
[evolutionary-patterns](https://github.com/generic-github-user/evolutionary-patterns/) |  |  | **[0](https://github.com/generic-github-user/evolutionary-patterns/issues)** [+](https://github.com/generic-github-user/evolutionary-patterns/issues/new) |  |  | Apr 2021 | 5 KB
[Alexandria](https://github.com/generic-github-user/Alexandria/) | Powerful, fast bookmark and web history management... |  | **[32](https://github.com/generic-github-user/Alexandria/issues)** [+](https://github.com/generic-github-user/Alexandria/issues/new) |  |  | Apr 2021 | 41 KB
[consequi](https://github.com/generic-github-user/consequi/) |  |  | **[30](https://github.com/generic-github-user/consequi/issues)** [+](https://github.com/generic-github-user/consequi/issues/new) | ✅ |  | Apr 2021 | 87 KB
[conlang](https://github.com/generic-github-user/conlang/) | Scripts & information for my (currently unnamed) c... | `python` `conlang` | **[0](https://github.com/generic-github-user/conlang/issues)** [+](https://github.com/generic-github-user/conlang/issues/new) | ✅ |  | Apr 2021 | 34 KB
[handwriting-generator](https://github.com/generic-github-user/handwriting-generator/) |  |  | **[0](https://github.com/generic-github-user/handwriting-generator/issues)** [+](https://github.com/generic-github-user/handwriting-generator/issues/new) | ✅ |  | Apr 2021 | 32 KB
[ascii-physics-sim](https://github.com/generic-github-user/ascii-physics-sim/) | A physics simulation written in Python that can re... | `python` `simulation` `ascii` `physics` | **[0](https://github.com/generic-github-user/ascii-physics-sim/issues)** [+](https://github.com/generic-github-user/ascii-physics-sim/issues/new) | ✅ |  | Apr 2021 | 82 KB
[ksp-tools](https://github.com/generic-github-user/ksp-tools/) | Some assorted tools for Kerbal Space Program |  | **[1](https://github.com/generic-github-user/ksp-tools/issues)** [+](https://github.com/generic-github-user/ksp-tools/issues/new) | ✅ |  | Jul 2020 | 2 KB
[PowerNote](https://github.com/generic-github-user/PowerNote/) | 💥 Version control and idea management for manics w... | `organization` `notes`*(3 more)* | **[41](https://github.com/generic-github-user/PowerNote/issues)** [+](https://github.com/generic-github-user/PowerNote/issues/new) | ✅ |  | Jun 2020 | 51 KB
[hackathon-2020](https://github.com/generic-github-user/hackathon-2020/) | StudentPulse | `education` `school` `communication`*(2 more)* | **[1](https://github.com/generic-github-user/hackathon-2020/issues)** [+](https://github.com/generic-github-user/hackathon-2020/issues/new) | ✅ | ✅ | Feb 2020 | 1537 KB
[snippets](https://github.com/generic-github-user/snippets/) | My collection of Atom snippets | `atom` `snippets` | **[0](https://github.com/generic-github-user/snippets/issues)** [+](https://github.com/generic-github-user/snippets/issues/new) | ✅ |  | Feb 2020 | 11 KB
[fire-spread-simulation](https://github.com/generic-github-user/fire-spread-simulation/) |  |  | **[0](https://github.com/generic-github-user/fire-spread-simulation/issues)** [+](https://github.com/generic-github-user/fire-spread-simulation/issues/new) |  |  | Jan 2020 | 6 KB
[fbla-competition-coding-programming](https://github.com/generic-github-user/fbla-competition-coding-programming/) | Entry for https://www.fbla-pbl.org/competitive-eve... | `electron` `javascript` `css` `html`*(1 more)* | **[0](https://github.com/generic-github-user/fbla-competition-coding-programming/issues)** [+](https://github.com/generic-github-user/fbla-competition-coding-programming/issues/new) | ✅ | ✅ | Jan 2020 | 1816 KB
[ideas](https://github.com/generic-github-user/ideas/) | List of ideas and stuff to do when i'm bored | `todo` `meta` `organization` `ideas` | **[125](https://github.com/generic-github-user/ideas/issues)** [+](https://github.com/generic-github-user/ideas/issues/new) | ✅ |  | Oct 2019 | 0 KB
[web-app-template](https://github.com/generic-github-user/web-app-template/) | A simple template repository for HTML web apps | `javascript` `css` `html` `web-app` | **[0](https://github.com/generic-github-user/web-app-template/issues)** [+](https://github.com/generic-github-user/web-app-template/issues/new) | ✅ |  | Oct 2019 | 0 KB
[metadata-tools](https://github.com/generic-github-user/metadata-tools/) | Some tools for using and changing Windows/NTFS fil... | `python` `metadata` `utilities` | **[12](https://github.com/generic-github-user/metadata-tools/issues)** [+](https://github.com/generic-github-user/metadata-tools/issues/new) | ✅ |  | Oct 2019 | 13 KB
[snippetcite](https://github.com/generic-github-user/snippetcite/) | Used a bunch of random code from StackOverflow and... |  | **[1](https://github.com/generic-github-user/snippetcite/issues)** [+](https://github.com/generic-github-user/snippetcite/issues/new) | ✅ |  | Oct 2019 | 1 KB
[Interact](https://github.com/generic-github-user/Interact/) | JavaScript tools for adding layout interactivity t... |  | **[0](https://github.com/generic-github-user/Interact/issues)** [+](https://github.com/generic-github-user/Interact/issues/new) | ✅ |  | Oct 2019 | 0 KB
[clocks](https://github.com/generic-github-user/clocks/) | A series of beautiful physics-based interactive wa... |  | **[0](https://github.com/generic-github-user/clocks/issues)** [+](https://github.com/generic-github-user/clocks/issues/new) | ✅ |  | Oct 2019 | 44 KB
[Recurrent-Neural-Illustration](https://github.com/generic-github-user/Recurrent-Neural-Illustration/) |  |  | **[0](https://github.com/generic-github-user/Recurrent-Neural-Illustration/issues)** [+](https://github.com/generic-github-user/Recurrent-Neural-Illustration/issues/new) | ✅ |  | Jun 2019 | 0 KB
[Meta-Learning](https://github.com/generic-github-user/Meta-Learning/) |  |  | **[0](https://github.com/generic-github-user/Meta-Learning/issues)** [+](https://github.com/generic-github-user/Meta-Learning/issues/new) | ✅ |  | Jun 2019 | 0 KB
[Floating-Tile-Grid](https://github.com/generic-github-user/Floating-Tile-Grid/) | After 8 hours of AI research, I would appreciate a... |  | **[3](https://github.com/generic-github-user/Floating-Tile-Grid/issues)** [+](https://github.com/generic-github-user/Floating-Tile-Grid/issues/new) | ✅ |  | Jun 2019 | 8611 KB
[MNIST](https://github.com/generic-github-user/MNIST/) |  |  | **[0](https://github.com/generic-github-user/MNIST/issues)** [+](https://github.com/generic-github-user/MNIST/issues/new) | ✅ |  | Jun 2019 | 0 KB
[Circle-Counter](https://github.com/generic-github-user/Circle-Counter/) | A simple TensorFlow.js project to count randomly g... |  | **[11](https://github.com/generic-github-user/Circle-Counter/issues)** [+](https://github.com/generic-github-user/Circle-Counter/issues/new) | ✅ |  | Jun 2019 | 38 KB
[Release](https://github.com/generic-github-user/Release/) | Automatically release your GitHub projects into th... |  | **[0](https://github.com/generic-github-user/Release/issues)** [+](https://github.com/generic-github-user/Release/issues/new) | ✅ |  | Jun 2019 | 0 KB
[The-Datalist](https://github.com/generic-github-user/The-Datalist/) | Mostly for online data (and data that cannot be ex... |  | **[1](https://github.com/generic-github-user/The-Datalist/issues)** [+](https://github.com/generic-github-user/The-Datalist/issues/new) | ✅ |  | Jun 2019 | 1 KB
[Neuroevolution-Benchmarks](https://github.com/generic-github-user/Neuroevolution-Benchmarks/) | Comparisons between neuroevolution/genetic algorit... | `deep-learning` `neuroevolution`*(1 more)* | **[14](https://github.com/generic-github-user/Neuroevolution-Benchmarks/issues)** [+](https://github.com/generic-github-user/Neuroevolution-Benchmarks/issues/new) | ✅ |  | Jun 2019 | 26 KB
[mineflayer-test](https://github.com/generic-github-user/mineflayer-test/) | Tests of the Prismarine JS Mineflayer bot system f... | `bot` `minecraft` `prismarine`*(1 more)* | **[10](https://github.com/generic-github-user/mineflayer-test/issues)** [+](https://github.com/generic-github-user/mineflayer-test/issues/new) | ✅ |  | May 2019 | 23 KB
[Structure-NBT-Decoder](https://github.com/generic-github-user/Structure-NBT-Decoder/) | Decoding and reorganizing data from structures cre... | `javascript` `minecraft` `json` `nbt`*(2 more)* | **[1](https://github.com/generic-github-user/Structure-NBT-Decoder/issues)** [+](https://github.com/generic-github-user/Structure-NBT-Decoder/issues/new) | ✅ |  | May 2019 | 11 KB
[WeatherWear](https://github.com/generic-github-user/WeatherWear/) |  | `weather` `app` `utility` `clothing`*(1 more)* | **[2](https://github.com/generic-github-user/WeatherWear/issues)** [+](https://github.com/generic-github-user/WeatherWear/issues/new) | ✅ |  | Feb 2019 | 1188 KB
[Delphi](https://github.com/generic-github-user/Delphi/) | Experiments in natural language interfaces and gen... |  | **[0](https://github.com/generic-github-user/Delphi/issues)** [+](https://github.com/generic-github-user/Delphi/issues/new) | ✅ |  | Feb 2019 | 0 KB
[icon-encoder](https://github.com/generic-github-user/icon-encoder/) | An autoencoder for website favicons, implemented i... | `machine-learning`*(3 more)* | **[12](https://github.com/generic-github-user/icon-encoder/issues)** [+](https://github.com/generic-github-user/icon-encoder/issues/new) | ✅ |  | Feb 2019 | 2586 KB
[Crowd-Dynamics-Simulation](https://github.com/generic-github-user/Crowd-Dynamics-Simulation/) | A mathematical simulation of static crowd dynamics... | `javascript` `web` `simulation`*(1 more)* | **[0](https://github.com/generic-github-user/Crowd-Dynamics-Simulation/issues)** [+](https://github.com/generic-github-user/Crowd-Dynamics-Simulation/issues/new) | ✅ |  | Jan 2019 | 6 KB
[Discord-Server-Archives](https://github.com/generic-github-user/Discord-Server-Archives/) | Publicly accessible data from Discord server expor... | `export` `backup` `discord` `archive`*(1 more)* | **[1](https://github.com/generic-github-user/Discord-Server-Archives/issues)** [+](https://github.com/generic-github-user/Discord-Server-Archives/issues/new) | ✅ |  | Jan 2019 | 3796 KB
[Time-Travel-Simulation](https://github.com/generic-github-user/Time-Travel-Simulation/) |  |  | **[0](https://github.com/generic-github-user/Time-Travel-Simulation/issues)** [+](https://github.com/generic-github-user/Time-Travel-Simulation/issues/new) | ✅ |  | Jan 2019 | 0 KB
[Time-Zone-Roles](https://github.com/generic-github-user/Time-Zone-Roles/) | A Discord bot for international servers that displ... | `javascript` `discord` `timezone`*(4 more)* | **[23](https://github.com/generic-github-user/Time-Zone-Roles/issues)** [+](https://github.com/generic-github-user/Time-Zone-Roles/issues/new) | ✅ |  | Jan 2019 | 64 KB
[Meta](https://github.com/generic-github-user/Meta/) | Information about me and my repositories. |  | **[0](https://github.com/generic-github-user/Meta/issues)** [+](https://github.com/generic-github-user/Meta/issues/new) | ✅ |  | Jan 2019 | 0 KB
[Web-Page-in-URL](https://github.com/generic-github-user/Web-Page-in-URL/) |  | `javascript` `url` `html` `browser` `web` | **[1](https://github.com/generic-github-user/Web-Page-in-URL/issues)** [+](https://github.com/generic-github-user/Web-Page-in-URL/issues/new) | ✅ |  | Jan 2019 | 11 KB
[RotationArt](https://github.com/generic-github-user/RotationArt/) |  |  | **[0](https://github.com/generic-github-user/RotationArt/issues)** [+](https://github.com/generic-github-user/RotationArt/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Number-Generator](https://github.com/generic-github-user/Number-Generator/) | A calculator for generating a number of different ... |  | **[0](https://github.com/generic-github-user/Number-Generator/issues)** [+](https://github.com/generic-github-user/Number-Generator/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Pong](https://github.com/generic-github-user/Pong/) | Simple JavaScript Pong clone. | `javascript` `game` `web` `pong` | **[0](https://github.com/generic-github-user/Pong/issues)** [+](https://github.com/generic-github-user/Pong/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Fireworks](https://github.com/generic-github-user/Fireworks/) |  | `javascript` `creative-coding`*(2 more)* | **[0](https://github.com/generic-github-user/Fireworks/issues)** [+](https://github.com/generic-github-user/Fireworks/issues/new) | ✅ |  | Jan 2019 | 96 KB
[Random-Equation-Generator](https://github.com/generic-github-user/Random-Equation-Generator/) |  |  | **[0](https://github.com/generic-github-user/Random-Equation-Generator/issues)** [+](https://github.com/generic-github-user/Random-Equation-Generator/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Fizzbuzz](https://github.com/generic-github-user/Fizzbuzz/) |  |  | **[0](https://github.com/generic-github-user/Fizzbuzz/issues)** [+](https://github.com/generic-github-user/Fizzbuzz/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Scribble](https://github.com/generic-github-user/Scribble/) |  |  | **[0](https://github.com/generic-github-user/Scribble/issues)** [+](https://github.com/generic-github-user/Scribble/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Platforming-Game](https://github.com/generic-github-user/Platforming-Game/) | One of my first JavaScript/HTML games: a simple en... | `javascript` `game`*(2 more)* | **[1](https://github.com/generic-github-user/Platforming-Game/issues)** [+](https://github.com/generic-github-user/Platforming-Game/issues/new) | ✅ |  | Jan 2019 | 4 KB
[Chaos-Game-Fractals](https://github.com/generic-github-user/Chaos-Game-Fractals/) |  |  | **[0](https://github.com/generic-github-user/Chaos-Game-Fractals/issues)** [+](https://github.com/generic-github-user/Chaos-Game-Fractals/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Plasma](https://github.com/generic-github-user/Plasma/) |  |  | **[0](https://github.com/generic-github-user/Plasma/issues)** [+](https://github.com/generic-github-user/Plasma/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Prime-Numbers](https://github.com/generic-github-user/Prime-Numbers/) |  |  | **[0](https://github.com/generic-github-user/Prime-Numbers/issues)** [+](https://github.com/generic-github-user/Prime-Numbers/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Dust](https://github.com/generic-github-user/Dust/) |  |  | **[0](https://github.com/generic-github-user/Dust/issues)** [+](https://github.com/generic-github-user/Dust/issues/new) | ✅ |  | Jan 2019 | 2 KB
[Dice](https://github.com/generic-github-user/Dice/) | Simulation of normal distributions by rolling many... |  | **[0](https://github.com/generic-github-user/Dice/issues)** [+](https://github.com/generic-github-user/Dice/issues/new) | ✅ |  | Jan 2019 | 0 KB
[Bezier-Rainbow](https://github.com/generic-github-user/Bezier-Rainbow/) |  |  | **[0](https://github.com/generic-github-user/Bezier-Rainbow/issues)** [+](https://github.com/generic-github-user/Bezier-Rainbow/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Galaxy](https://github.com/generic-github-user/Galaxy/) |  |  | **[0](https://github.com/generic-github-user/Galaxy/issues)** [+](https://github.com/generic-github-user/Galaxy/issues/new) | ✅ |  | Jan 2019 | 11 KB
[Waves](https://github.com/generic-github-user/Waves/) |  |  | **[0](https://github.com/generic-github-user/Waves/issues)** [+](https://github.com/generic-github-user/Waves/issues/new) | ✅ |  | Jan 2019 | 180 KB
[Quadratic-Equation-Art](https://github.com/generic-github-user/Quadratic-Equation-Art/) |  |  | **[0](https://github.com/generic-github-user/Quadratic-Equation-Art/issues)** [+](https://github.com/generic-github-user/Quadratic-Equation-Art/issues/new) | ✅ |  | Jan 2019 | 1 KB
[Cursor](https://github.com/generic-github-user/Cursor/) |  |  | **[0](https://github.com/generic-github-user/Cursor/issues)** [+](https://github.com/generic-github-user/Cursor/issues/new) | ✅ | ✅ | Jan 2019 | 2 KB
[Mandelbrot](https://github.com/generic-github-user/Mandelbrot/) |  |  | **[0](https://github.com/generic-github-user/Mandelbrot/issues)** [+](https://github.com/generic-github-user/Mandelbrot/issues/new) | ✅ | ✅ | Jan 2019 | 6 KB
[Rainbow-Matrix](https://github.com/generic-github-user/Rainbow-Matrix/) |  |  | **[0](https://github.com/generic-github-user/Rainbow-Matrix/issues)** [+](https://github.com/generic-github-user/Rainbow-Matrix/issues/new) | ✅ | ✅ | Jan 2019 | 2 KB
[Neural-Cellular-Automata](https://github.com/generic-github-user/Neural-Cellular-Automata/) | Neural network-based cellular automata experiments... | `neural-network`*(1 more)* | **[8](https://github.com/generic-github-user/Neural-Cellular-Automata/issues)** [+](https://github.com/generic-github-user/Neural-Cellular-Automata/issues/new) | ✅ | ✅ | Dec 2018 | 2411 KB
[Alien-Invasion](https://github.com/generic-github-user/Alien-Invasion/) | Python example game from Python Crash Course. |  | **[0](https://github.com/generic-github-user/Alien-Invasion/issues)** [+](https://github.com/generic-github-user/Alien-Invasion/issues/new) |  |  | Dec 2018 | 397 KB
[Random-Wikipedia-Aritlce](https://github.com/generic-github-user/Random-Wikipedia-Aritlce/) |  |  | **[0](https://github.com/generic-github-user/Random-Wikipedia-Aritlce/issues)** [+](https://github.com/generic-github-user/Random-Wikipedia-Aritlce/issues/new) | ✅ | ✅ | Dec 2018 | 1 KB
[Bubbles](https://github.com/generic-github-user/Bubbles/) | Draw with animated bubbles. |  | **[0](https://github.com/generic-github-user/Bubbles/issues)** [+](https://github.com/generic-github-user/Bubbles/issues/new) | ✅ | ✅ | Dec 2018 | 4 KB
[WordData](https://github.com/generic-github-user/WordData/) | Analyze sets of text to draw comparisons, statisti... |  | **[1](https://github.com/generic-github-user/WordData/issues)** [+](https://github.com/generic-github-user/WordData/issues/new) | ✅ | ✅ | Dec 2018 | 2 KB
[XMLtoJSON](https://github.com/generic-github-user/XMLtoJSON/) | Convert an XML document to a JSON data tree struct... |  | **[0](https://github.com/generic-github-user/XMLtoJSON/issues)** [+](https://github.com/generic-github-user/XMLtoJSON/issues/new) | ✅ | ✅ | Dec 2018 | 1 KB
[ML-Projects](https://github.com/generic-github-user/ML-Projects/) | A list of machine learning and artificial intellig... | `machine-learning`*(1 more)* | **[0](https://github.com/generic-github-user/ML-Projects/issues)** [+](https://github.com/generic-github-user/ML-Projects/issues/new) | ✅ | ✅ | Dec 2018 | 2 KB
[Blender](https://github.com/generic-github-user/Blender/) | Scripts and add-ons for the Blender 3D Python inte... | `python` `blender-scripts` `blender`*(3 more)* | **[0](https://github.com/generic-github-user/Blender/issues)** [+](https://github.com/generic-github-user/Blender/issues/new) | ✅ | ✅ | Dec 2018 | 30 KB
[Function-Remover](https://github.com/generic-github-user/Function-Remover/) | Take that nice, organized JavaScript code and easi... |  | **[0](https://github.com/generic-github-user/Function-Remover/issues)** [+](https://github.com/generic-github-user/Function-Remover/issues/new) | ✅ | ✅ | Dec 2018 | 0 KB
[Neural-Turing-Machine](https://github.com/generic-github-user/Neural-Turing-Machine/) | A neural turing machine implementation using Tenso... | `machine-learning`*(4 more)* | **[3](https://github.com/generic-github-user/Neural-Turing-Machine/issues)** [+](https://github.com/generic-github-user/Neural-Turing-Machine/issues/new) | ✅ | ✅ | Dec 2018 | 259 KB
[Bug-Tracker](https://github.com/generic-github-user/Bug-Tracker/) | A semi-accurate simulation of what it's like tryin... |  | **[0](https://github.com/generic-github-user/Bug-Tracker/issues)** [+](https://github.com/generic-github-user/Bug-Tracker/issues/new) | ✅ | ✅ | Dec 2018 | 5 KB
[Tree](https://github.com/generic-github-user/Tree/) | A web-based viewer for file tree structures. | `visualization` `javascript`*(2 more)* | **[2](https://github.com/generic-github-user/Tree/issues)** [+](https://github.com/generic-github-user/Tree/issues/new) | ✅ | ✅ | Dec 2018 | 2552 KB
[Cellular-Automata](https://github.com/generic-github-user/Cellular-Automata/) | Cellular automata simulation demos in JavaScript |  | **[0](https://github.com/generic-github-user/Cellular-Automata/issues)** [+](https://github.com/generic-github-user/Cellular-Automata/issues/new) | ✅ | ✅ | Dec 2018 | 28 KB
[Rainbow-Text-Generator](https://github.com/generic-github-user/Rainbow-Text-Generator/) | Create rainbow-colored text that you can copy and ... |  | **[0](https://github.com/generic-github-user/Rainbow-Text-Generator/issues)** [+](https://github.com/generic-github-user/Rainbow-Text-Generator/issues/new) | ✅ | ✅ | Dec 2018 | 3 KB
[Binary-Clock](https://github.com/generic-github-user/Binary-Clock/) | A creative coding experiment |  | **[0](https://github.com/generic-github-user/Binary-Clock/issues)** [+](https://github.com/generic-github-user/Binary-Clock/issues/new) | ✅ |  | Dec 2018 | 1 KB
[Tsunami](https://github.com/generic-github-user/Tsunami/) | JavaScript neural networks |  | **[0](https://github.com/generic-github-user/Tsunami/issues)** [+](https://github.com/generic-github-user/Tsunami/issues/new) | ✅ | ✅ | Nov 2018 | 339 KB
[Hurricane](https://github.com/generic-github-user/Hurricane/) | Outdated JavaScript neural network library. | `javascript` `machine-learning`*(2 more)* | **[0](https://github.com/generic-github-user/Hurricane/issues)** [+](https://github.com/generic-github-user/Hurricane/issues/new) | ✅ | ✅ | Nov 2018 | 168 KB
[Bookmarklets](https://github.com/generic-github-user/Bookmarklets/) | JavaScript bookmarklets that can be used to execut... |  | **[3](https://github.com/generic-github-user/Bookmarklets/issues)** [+](https://github.com/generic-github-user/Bookmarklets/issues/new) | ✅ |  | Nov 2018 | 3 KB
[Neural-Network-Demo](https://github.com/generic-github-user/Neural-Network-Demo/) | A visualization of a simple neural network. |  | **[0](https://github.com/generic-github-user/Neural-Network-Demo/issues)** [+](https://github.com/generic-github-user/Neural-Network-Demo/issues/new) | ✅ | ✅ | Nov 2018 | 0 KB
[Supernova](https://github.com/generic-github-user/Supernova/) | JavaScript neural network library |  | **[0](https://github.com/generic-github-user/Supernova/issues)** [+](https://github.com/generic-github-user/Supernova/issues/new) | ✅ |  | Nov 2018 | 71 KB
[Float](https://github.com/generic-github-user/Float/) | Interactive animated background effects for the we... |  | **[0](https://github.com/generic-github-user/Float/issues)** [+](https://github.com/generic-github-user/Float/issues/new) | ✅ |  | Nov 2018 | 0 KB
[Web](https://github.com/generic-github-user/Web/) | Creative visualization of a procedurally generated... |  | **[0](https://github.com/generic-github-user/Web/issues)** [+](https://github.com/generic-github-user/Web/issues/new) | ✅ |  | Nov 2018 | 5 KB
[Textgrab](https://github.com/generic-github-user/Textgrab/) | Get text content from any URL. |  | **[0](https://github.com/generic-github-user/Textgrab/issues)** [+](https://github.com/generic-github-user/Textgrab/issues/new) | ✅ |  | Oct 2018 | 65 KB
[Hashtagifier](https://github.com/generic-github-user/Hashtagifier/) | Easily make all your social media posts as annoyin... | `hashtag` `social-media` | **[7](https://github.com/generic-github-user/Hashtagifier/issues)** [+](https://github.com/generic-github-user/Hashtagifier/issues/new) | ✅ |  | Oct 2018 | 30125 KB
[Image-Convolution-Playground](https://github.com/generic-github-user/Image-Convolution-Playground/) | Experiment with image convolutions in your browser... | `images` `mathematics`*(2 more)* | **[12](https://github.com/generic-github-user/Image-Convolution-Playground/issues)** [+](https://github.com/generic-github-user/Image-Convolution-Playground/issues/new) | ✅ | ✅ | Sep 2018 | 1326 KB
[random-walk](https://github.com/generic-github-user/random-walk/) | A simple random walk, implemented in JavaScript. |  | **[0](https://github.com/generic-github-user/random-walk/issues)** [+](https://github.com/generic-github-user/random-walk/issues/new) | ✅ | ✅ | Aug 2018 | 2 KB
[micronotes](https://github.com/generic-github-user/micronotes/) | Nice-looking, simple to use JavaScript footnotes. | `javascript` `plugin` `utility` `widget`*(2 more)* | **[16](https://github.com/generic-github-user/micronotes/issues)** [+](https://github.com/generic-github-user/micronotes/issues/new) | ✅ | ✅ | Aug 2018 | 239 KB
[Caesium](https://github.com/generic-github-user/Caesium/) | General-purpose AI library with NEAT-style genetic... | `ai` `neat` `neuroevolution`*(5 more)* | **[52](https://github.com/generic-github-user/Caesium/issues)** [+](https://github.com/generic-github-user/Caesium/issues/new) | ✅ | ✅ | Aug 2018 | 3718 KB
[Hyperblock](https://github.com/generic-github-user/Hyperblock/) | A fun game. | `javascript` `game` `html` `web-game` | **[27](https://github.com/generic-github-user/Hyperblock/issues)** [+](https://github.com/generic-github-user/Hyperblock/issues/new) | ✅ |  | Jul 2018 | 26 KB
[Star](https://github.com/generic-github-user/Star/) | A website where anyone can rate anything. | `website` `app` `review` `social-media`*(1 more)* | **[45](https://github.com/generic-github-user/Star/issues)** [+](https://github.com/generic-github-user/Star/issues/new) | ✅ |  | Jul 2018 | 890 KB
[Arduino](https://github.com/generic-github-user/Arduino/) | A set of Arduino tests and experiments. | `arduino` | **[6](https://github.com/generic-github-user/Arduino/issues)** [+](https://github.com/generic-github-user/Arduino/issues/new) | ✅ |  | Jul 2018 | 8 KB
[tfjs-visual-editor](https://github.com/generic-github-user/tfjs-visual-editor/) | A visual interface for TensorFlow.js code | `javascript` `css` `html` `data`*(5 more)* | **[47](https://github.com/generic-github-user/tfjs-visual-editor/issues)** [+](https://github.com/generic-github-user/tfjs-visual-editor/issues/new) | ✅ | ✅ | Jul 2018 | 49 KB
[worst-syntax-ever](https://github.com/generic-github-user/worst-syntax-ever/) | Literally the worst syntax ever. |  | **[21](https://github.com/generic-github-user/worst-syntax-ever/issues)** [+](https://github.com/generic-github-user/worst-syntax-ever/issues/new) | ✅ | ✅ | Jul 2018 | 141 KB
[xkcd-Generator](https://github.com/generic-github-user/xkcd-Generator/) | TensorFlow.js GAN for generating new xkcd comics | `javascript` `machine-learning`*(2 more)* | **[22](https://github.com/generic-github-user/xkcd-Generator/issues)** [+](https://github.com/generic-github-user/xkcd-Generator/issues/new) | ✅ | ✅ | Jul 2018 | 1154 KB
[Deep-Dream](https://github.com/generic-github-user/Deep-Dream/) | Deep dreaming images with TensorFlow.js | `javascript` `machine-learning`*(2 more)* | **[6](https://github.com/generic-github-user/Deep-Dream/issues)** [+](https://github.com/generic-github-user/Deep-Dream/issues/new) | ✅ |  | Jul 2018 | 2333 KB
[TensorFlow.js-VAE](https://github.com/generic-github-user/TensorFlow.js-VAE/) | A variational autoencoder for TensorFlow.js | `javascript` `machine-learning`*(3 more)* | **[12](https://github.com/generic-github-user/TensorFlow.js-VAE/issues)** [+](https://github.com/generic-github-user/TensorFlow.js-VAE/issues/new) | ✅ |  | Jun 2018 | 453 KB

In [ ]:
x = [print(repo['name']) for repo in response]